In [ ]:
I have a parquet file with the following schema:
        schema = pa.schema([
            ('train_SRC', pa.string()),
            ('cleaned_pizza_orders_src', pa.string()),
            ('cleaned_drink_orders_src', pa.string())
        ])
train_SRC is an order
cleaned_pizza_orders_src is a list of pizza suborders
cleaned_drink_orders_src is a list of drink suborders

I need to train a model to be able to extract the pizza and drink suborders from the order.

Below is a preprocessing function


import pandas as pd
import numpy as np
import pyarrow as pa

# Example of the input data schema (you would use your actual dataset)
data = [
    {
        'train_SRC': "one large pepperoni pizza and balsamic glaze pizza and a coke and 3 spirits",
        'cleaned_pizza_orders_src': "one large pepperoni pizza, balsamic glaze pizza",
        'cleaned_drink_orders_src': "a coke, 3 spirits"
    },
    # Add more rows as needed...
]

df = pd.DataFrame(data)

# Preprocess the data to create tokenized input and labels
def preprocess_data(df):
    X_tokenized = []  # List of tokenized input sentences
    y_labels = []     # Corresponding labels (pizza, drink, or neither)
    
    for index, row in df.iterrows():
        input_text = row['train_SRC']
        pizza_text = row['cleaned_pizza_orders_src']
        drink_text = row['cleaned_drink_orders_src']

        # Tokenize the input sentence
        tokens = input_text.lower().split()

        # Create labels for each token: 0 for neither, 1 for pizza, 2 for drink
        labels = []
        pizza_tokens = pizza_text.lower().split()
        drink_tokens = drink_text.lower().split()

        pizza_in_order = False
        drink_in_order = False
        last_token_type = None  # None means no order detected yet

        # Pizza: Start with 1, continue with 2, end with 3
        # Drink: Start with 4, continue with 5, end with 6
        # Neither: 0

        for token in tokens:
            if token in pizza_tokens:

                if last_token_type == 'pizza':
                    # Continue the pizza order
                    labels.append(2)
                else:
                    # Is it a new pizza order? If so, end the previous drink order
                    if last_token_type == 'drink':
                        labels.append(6)
                    # Start new pizza order
                    labels.append(1)
                    # Append the token
                    labels.append(2)
                    last_token_type = 'pizza'
                
            elif token in drink_tokens:
                if last_token_type == 'drink':
                    # Continue the drink order
                    labels.append(5)
                else:
                    # Is it a new drink order? If so, end the previous pizza order
                    if last_token_type == 'pizza':
                        labels.append(3)
                    # Start new drink order
                    labels.append(4)
                    # Append the token
                    labels.append(5)
                    last_token_type = 'drink'

            else:
                if last_token_type == 'pizza':
                    labels.append(3)
                elif last_token_type == 'drink':
                    labels.append(6)
                labels.append(0)
                last_token_type = None             

    return X_tokenized, y_labels

# Apply preprocessing to the dataset
X_tokenized, y_labels = preprocess_data(df)

# Output the processed data with <START> and <END>
for i in range(len(X_tokenized)):
    print(f"Input: {X_tokenized[i]}")
    print(f"Labels: {y_labels[i]}")


In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa

# Example of the input data schema (you would use your actual dataset)
data = [
    {
        'train_SRC': "one large pepperoni pizza and balsamic glaze pizza and a coke and 3 spirits",
        'cleaned_pizza_orders_src': "one large pepperoni pizza, balsamic glaze pizza",
        'cleaned_drink_orders_src': "a coke, 3 spirits"
    },
    # Add more rows as needed...
]

df = pd.DataFrame(data)

# Preprocess the data to create tokenized input and labels
def preprocess_data(df):
    X_tokenized = []  # List of tokenized input sentences
    y_labels = []     # Corresponding labels (pizza, drink, or neither)
    
    for index, row in df.iterrows():
        input_text = row['train_SRC']
        pizza_text = row['cleaned_pizza_orders_src']
        drink_text = row['cleaned_drink_orders_src']

        # Tokenize the input sentence
        tokens = input_text.lower().split()

        # Create labels for each token: 0 for neither, 1 for pizza, 2 for drink
        labels = []
        pizza_tokens = pizza_text.lower().split()
        drink_tokens = drink_text.lower().split()

        pizza_in_order = False
        drink_in_order = False
        last_token_type = None  # None means no order detected yet

        for token in tokens:
            if token in pizza_tokens:
                if last_token_type == 'pizza':
                    labels.append(2)  # Continue the pizza order
                else:
                    if last_token_type == 'drink':  # End the previous drink order
                        labels.append(6)
                    labels.append(1)  # Start new pizza order
                    labels.append(2)  # Inside pizza order
                    last_token_type = 'pizza'

            elif token in drink_tokens:
                if last_token_type == 'drink':
                    labels.append(5)  # Continue the drink order
                else:
                    if last_token_type == 'pizza':  # End the previous pizza order
                        labels.append(3)
                    labels.append(4)  # Start new drink order
                    labels.append(5)  # Inside drink order
                    last_token_type = 'drink'

            else:
                if last_token_type == 'pizza':
                    labels.append(3)  # End the pizza order
                elif last_token_type == 'drink':
                    labels.append(6)  # End the drink order
                labels.append(0)  # Neither order
                last_token_type = None

        X_tokenized.append(tokens)
        y_labels.append(labels)

    return X_tokenized, y_labels

# Apply preprocessing to the dataset
X_tokenized, y_labels = preprocess_data(df)

# Output the processed data with <START> and <END>
for i in range(len(X_tokenized)):
    print(f"Input: {X_tokenized[i]}")
    print(f"Labels: {y_labels[i]}")
